In [1]:
import requests, pandas as pd, numpy as np
from requests import session
from bs4 import BeautifulSoup

In [2]:
manual=pd.read_excel('manual_manual.xlsx').set_index('Cégnév')
data=pd.read_excel('data.xlsx').set_index('Cégnév')
sectors=pd.read_excel('valid_manual.xlsx')

In [3]:
len(data), len(manual)

(422, 2)

In [4]:
data=pd.concat([data,manual])
data['Sector']=sectors.set_index(0)[1]

Bring in 2014 data

In [5]:
manual4=pd.read_excel('../2018/manual_manual.xlsx').set_index('Cégnév')
data4=pd.read_excel('../2018/data.xlsx').set_index('Cégnév')
sectors4=pd.read_excel('../2018/valid_manual.xlsx')

In [6]:
len(data4), len(manual4)

(414, 10)

In [7]:
data4=pd.concat([data4,manual4])
data4['Sector4']=sectors4.set_index(0)[1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
data=data.join(data4[['Alkalmazottak száma 2014','Árbevétel 2014 (RON)']])

In [9]:
repl={'udvarhelyszek':'Udvarhelyszék',
'csikszek':'Csíkszék',
'gyergyoszek':'Gyergyószék',
'marosszek':'Marosszék',
'also-haromszek':'Alsó-háromszék',
'felso-haromszek':'Felső-háromszék'}
for r in repl:
    data=data.replace(r,repl[r])

In [10]:
data.to_excel('export1.xlsx')

In [11]:
d1=data[['Régió', 'Latitude', 'Longitude', 'Sector']]
d1.columns=['Régió', 'Hosszúság', 'Szélesség', 'Iparág']

In [12]:
d1.to_excel('d1.xlsx')

In [13]:
data.columns

Index(['Unnamed: 0', 'tavaly_left', 'Alkalmazottak száma 2018',
       'Alkalmazottak száma 2017', 'Alkalmazottak száma 2016',
       'Alkalmazottak száma 2015', 'nr_alkalmazottak', 'tavaly_right',
       'Árbevétel 2018 (RON)', 'Árbevétel 2017 (RON)', 'Árbevétel 2016 (RON)',
       'Árbevétel 2015 (RON)', 'nr_arbevetel', 'Cím', 'Kw', 'Régió',
       'Latitude', 'Longitude', 'Sector', 'Alkalmazottak száma 2014',
       'Árbevétel 2014 (RON)'],
      dtype='object')

In [14]:
ds=[]
ds1=[]
ds2=[]
for i in range(2014,2019):
    g='Alkalmazottak száma'
    c=g+' '+str(i)
    print(c)
    df=data[[c]]
    df.columns=[g]
    df[g]=df[g].fillna('').str.replace(u'\xa0','').str.replace('-','')
    df['Év']=i
    ds1.append(df)
    ds.append(d1.join(df))
    g='Árbevétel'
    c=g+' '+str(i)+' (RON)'
    print(c)
    df=data[[c]]
    df.columns=[g]
    df[g]=df[g].fillna('').str.replace(u'\xa0','').str.replace('-','')
    df['Év']=i
    ds2.append(df)

Alkalmazottak száma 2014
Árbevétel 2014 (RON)
Alkalmazottak száma 2015
Árbevétel 2015 (RON)
Alkalmazottak száma 2016
Árbevétel 2016 (RON)
Alkalmazottak száma 2017
Árbevétel 2017 (RON)
Alkalmazottak száma 2018
Árbevétel 2018 (RON)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [15]:
df=pd.concat(ds,sort=False)

In [16]:
df1=pd.concat(ds1,sort=False)
df2=pd.concat(ds2,sort=False)

In [17]:
df1.to_excel('df1.xlsx')
df2.to_excel('df2.xlsx')

In [18]:
df1=df1.set_index('Év',append=True)
df2=df2.set_index('Év',append=True)

In [19]:
df=df1.join(df2).replace('',np.nan).astype(float)

In [20]:
df['Bevétel']=np.round(df['Árbevétel']/1000000.0,3)

In [21]:
df['Bevétel/Alkalmazott']=np.round(df['Bevétel']/df['Alkalmazottak száma'],3)

In [22]:
data2=df.reset_index().join(d1,on='Cégnév').set_index('Cégnév')

Normalize names

In [23]:
#[' '.join([j.title() if len(j)>3 else j for j in i.split(' ')]) for i in data2.index.str.replace(' SA','').str.replace(' SRL','').str.replace(' ROMANIA ','')]
data2.index=data2.index.str.replace(' SA','').str.replace(' SRL','').str.replace(' ROMANIA ','')

In [24]:
data2.to_excel('export2.xlsx')

In [25]:
data3=df.unstack()
data3.columns=data3.columns.get_level_values(1).astype(str)
data3=data3.reset_index().join(d1,on='Cégnév').set_index('Cégnév')
data3.index.name=None
data3.index=data3.index.str.replace(' SA','').str.replace(' SRL','').str.replace(' ROMANIA ','')
data3.to_excel('export3.xlsx')

In [26]:
data4=data3.groupby(['Iparág','Régió']).sum()
data4b=data3.groupby(['Régió']).sum()
data4=data4.unstack().T
data4b=data4b.unstack().T
data4['Mind']=data4b
data4=data4.unstack().T
data4.reset_index().to_excel('export4.xlsx')

In [27]:
data4=data3.groupby(['Régió','Iparág']).sum()
data4b=data3.groupby(['Iparág']).sum()
data4=data4.unstack().T
data4b=data4b.unstack().T
data4['Mind']=data4b
data4=data4.unstack().T
data4.reset_index().to_excel('export4b.xlsx')

In [34]:
d1s=[]
for y in ['2014','2015','2016','2017','2018']:
    d1=data4[[y]]
    d1.columns=[0,1,'Bevétel',2]
    d1=d1[['Bevétel']]
    d1['Év']=y
    d1s.append(d1)
d1s=pd.concat(d1s)
d1s.reset_index().to_excel('export5.xlsx')

In [100]:
d2s=d1s.reset_index().set_index(['Iparág','Év','Régió']).unstack()
d3s=d2s.reset_index().groupby(['Év']).sum()
d3s['Iparág']='Mind'
d3s=d3s.reset_index().set_index(['Iparág','Év'])
pd.concat([d2s,d3s]).reset_index().to_excel('export6.xlsx')